#算法代码及解析
（收集数据与算法测试阶段与同学共享数据、分享方法，但无共享代码，可能有部分雷同；小程序制作完全独立）

##数据处理
1、收集数据后导出json文件
2、筛选数据：算法所用数据频率为60ms/次，因不同手机稳定性不同，根据数据个数筛除差距较大数据
3、滤波：让数据更加平滑，减少手部不稳定带来的误差
4、拆分数据：每个窗格为2.56秒，采用两个窗格间重合一半的原则把数据拆分
5、每个窗格分别对x, y, z轴加速度，x, y, z轴陀螺仪数据提取六个特征值：均值、方差、最大值、最小值、中位数、绝对中位差，再把六组特征值合并，得到此窗格的36个特征值

In [ ]:
import json
from scipy import signal
import numpy as np
import joblib
from scipy import stats
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

#读取json文件
def read_json(file_name):
     with open(file_name,'r', encoding = 'utf-8') as file:
          data = []
          for line in file.readlines():
               dic = json.loads(line)
               data.append(dic)
     return data

#滤波
def lvbo(data):
     b, a = signal.butter(8, 0.1, 'lowpass')
     filtedData = signal.filtfilt(b, a, data)
     return filtedData

#把数据拆分为重叠的窗格
def dataOverlap(data):
    data2 = []
    length = len(data)
    n = length//21
    for i in range(n-1):
        data2.append([])
        for j in range(21*i,21*(i+2)):
            data2[i].append(data[j])
    return data2

#提取特征值
def feature(accx, accy, accz, gyrx, gyry, gyrz):
    data = []
    for i in range(len(accx)):
        data_ = []
        data_ += [max(accx[i]), min(accx[i]), np.median(accx[i]), np.mean(accx[i]), np.std(accx[i]), stats.median_absolute_deviation(accx[i])]
        data_ += [max(accy[i]), min(accy[i]), np.median(accy[i]), np.mean(accy[i]), np.std(accy[i]), stats.median_absolute_deviation(accy[i])]
        data_ += [max(accz[i]), min(accz[i]), np.median(accz[i]), np.mean(accz[i]), np.std(accz[i]), stats.median_absolute_deviation(accz[i])]
        data_ += [max(gyrx[i]), min(gyrx[i]), np.median(gyrx[i]), np.mean(gyrx[i]), np.std(gyrx[i]), stats.median_absolute_deviation(gyrx[i])]
        data_ += [max(gyry[i]), min(gyry[i]), np.median(gyry[i]), np.mean(gyry[i]), np.std(gyry[i]), stats.median_absolute_deviation(gyry[i])]
        data_ += [max(gyrz[i]), min(gyrz[i]), np.median(gyrz[i]), np.mean(gyrz[i]), np.std(gyrz[i]), stats.median_absolute_deviation(gyrz[i])]
        data.append(data_)
    return data

data = np.loadtxt('raw_data/data.txt')
target = []
for i in range(91):
    target.append(1)
for i in range(91, 198):
    target.append(2)
for i in range(198, 480):
    target.append(3)
for i in range(480, 806):
    target.append(4)

##算法
通过比较knn、随机森林等分类算法，随机森林有较高的准确率

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)
model = RandomForestClassifier()
model = model.fit(X_train, y_train)
result = model.score(X_test, y_test)
joblib.dump(model, "./rfc2.pkl")
print(result)
print(len(X_test))

#服务器代码
小程序每2.56秒把数据发送到服务器，服务器经过滤波、提取特征值的数据处理后，利用训练的模型预测动作，并将动作编号发送给小程序

In [ ]:
from flask import Flask,request
app = Flask(__name__)
import joblib
import numpy as np
from scipy import signal, stats
@app.route('/',methods=['post','get'])

def index():
    fr = open('rfcModel.pkl', 'rb')
    inf = joblib.load(fr)
    fr.close()

    data =  request.get_json()
    processed_data = []
    b, a = signal.butter(8, 0.1, 'lowpass')
    processed_data += dataProcess(data['accXs'], b, a)
    processed_data += dataProcess(data['accYs'], b, a)
    processed_data += dataProcess(data['accZs'], b, a)
    processed_data += dataProcess(data['gyrXs'], b, a)
    processed_data += dataProcess(data['gyrYs'], b, a)
    processed_data += dataProcess(data['gyrZs'], b, a)
    predict = inf.predict([processed_data])
    return str(predict)

def dataProcess(data,b,a):
    data_ = signal.filtfilt(b, a, data)
    processed_data = [max(data_), min(data_), np.median(data_), np.mean(data_), np.var(data_),stats.median_absolute_deviation(data_)]
    return processed_data

if __name__ == '__main__':
    app.run(host='172.17.0.2', debug=True,port=8000)

